In [1]:
# Import libraries

import os
import pyspark
import re
from operator import add
from pyspark.sql.types import *
from pyspark.sql import functions 
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, concat, isnull, mode, isnan, count, lower, upper, sum, first, regexp_replace, concat_ws, year, hour, minute, month, to_date, to_timestamp
from pyspark.sql.window import Window
import pyspark.sql.functions as F
import pandas as pd
import numpy as np
from numpy import mean
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import plotly.express as px
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
import warnings
warnings.filterwarnings('ignore')

2024-04-10 15:54:18.761992: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-10 15:54:18.776299: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-10 15:54:19.065303: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-10 15:54:21.916481: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## HADOOP directory file reading

In [2]:
df = spark.read.format("csv").option("header","true").load('/user1/Motor_Vehicle_Collisions_-_Crashes.csv')

In [3]:
df.show(5)

24/04/10 15:54:48 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----------+----------+--------+--------+---------+----------+--------------------+--------------------+-----------------+--------------------+-------------------------+------------------------+-----------------------------+----------------------------+-------------------------+------------------------+--------------------------+-------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+------------+-------------------+-------------------+-------------------+-------------------+-------------------+
|CRASH DATE|CRASH TIME| BOROUGH|ZIP CODE| LATITUDE| LONGITUDE|            LOCATION|      ON STREET NAME|CROSS STREET NAME|     OFF STREET NAME|NUMBER OF PERSONS INJURED|NUMBER OF PERSONS KILLED|NUMBER OF PEDESTRIANS INJURED|NUMBER OF PEDESTRIANS KILLED|NUMBER OF CYCLIST INJURED|NUMBER OF CYCLIST KILLED|NUMBER OF MOTORIST INJURED|NUMBER OF MOTORIST KILLED|CONTRIBUTING FACTOR VE

In [4]:
df.printSchema()

root
 |-- CRASH DATE: string (nullable = true)
 |-- CRASH TIME: string (nullable = true)
 |-- BOROUGH: string (nullable = true)
 |-- ZIP CODE: string (nullable = true)
 |-- LATITUDE: string (nullable = true)
 |-- LONGITUDE: string (nullable = true)
 |-- LOCATION: string (nullable = true)
 |-- ON STREET NAME: string (nullable = true)
 |-- CROSS STREET NAME: string (nullable = true)
 |-- OFF STREET NAME: string (nullable = true)
 |-- NUMBER OF PERSONS INJURED: string (nullable = true)
 |-- NUMBER OF PERSONS KILLED: string (nullable = true)
 |-- NUMBER OF PEDESTRIANS INJURED: string (nullable = true)
 |-- NUMBER OF PEDESTRIANS KILLED: string (nullable = true)
 |-- NUMBER OF CYCLIST INJURED: string (nullable = true)
 |-- NUMBER OF CYCLIST KILLED: string (nullable = true)
 |-- NUMBER OF MOTORIST INJURED: string (nullable = true)
 |-- NUMBER OF MOTORIST KILLED: string (nullable = true)
 |-- CONTRIBUTING FACTOR VEHICLE 1: string (nullable = true)
 |-- CONTRIBUTING FACTOR VEHICLE 2: string (nu

## Preprocessing

#### Data Cleaning

In [5]:
# Drop irrelevant columns

df =  df.drop('CROSS STREET NAME', 'OFF STREET NAME', 'NUMBER OF PEDESTRIANS INJURED', 'NUMBER OF CYCLIST INJURED',
            'NUMBER OF CYCLIST KILLED', 'NUMBER OF MOTORIST INJURED', 'NUMBER OF MOTORIST KILLED',
            'CONTRIBUTING FACTOR VEHICLE 2', 'CONTRIBUTING FACTOR VEHICLE 3', 'CONTRIBUTING FACTOR VEHICLE 4',
            'CONTRIBUTING FACTOR VEHICLE 5', 'COLLISION_ID', 'VEHICLE TYPE CODE 2', 'VEHICLE TYPE CODE 3',
            'VEHICLE TYPE CODE 4', 'VEHICLE TYPE CODE 5', 'NUMBER OF PEDESTRIANS KILLED')

df = df.na.drop(how='any')

#df1.select([count(when(
#         isnan(col(each_col)) | \
#         (col(each_col) == "") | \
#         isnull(col(each_col)) | \
#         (lower(col(each_col)) == "null"), 1 \
#     )).alias(f"{each_col}") for each_col in df.columns]).show()

# CHANGE CRASH DATE FORMAT

df = df.withColumn('CRASH DATE', regexp_replace(df['CRASH DATE'],"\\/" , "-"))

# Create new date column with date & time

df = df.withColumn('DATE', concat_ws(' ', df['CRASH DATE'], df['CRASH TIME']))

# DATE to timestamp

df = df.withColumn('DATE', to_timestamp(df['DATE'], 'MM-dd-yyyy H:mm'))

df = df.withColumn('CRASH DATE', to_date(df['CRASH DATE'], 'MM-dd-yyyy'))

# Move DATE column to pisition 0

df = df.select('DATE', *[column for column in df.columns if column != 'DATE'])

# Sort by date

df = df.orderBy('DATE', 'CRASH TIME')

#Filter out 2012 December records

df = df.filter(~((col('DATE').substr(1, 4) == '2012') & (col('DATE').substr(6, 2) != '12')))

df = df.withColumn(
    'ZIP CODE', df['ZIP CODE'].cast('integer')).withColumn(
    'NUMBER OF PERSONS INJURED', 
    df['NUMBER OF PERSONS INJURED'].cast('integer')).withColumn(
    'NUMBER OF PERSONS KILLED', df['NUMBER OF PERSONS KILLED'].cast('integer'))

# Standarize categorical columns

df = df.withColumn(
    'CONTRIBUTING FACTOR VEHICLE 1', 
    upper(col('CONTRIBUTING FACTOR VEHICLE 1'))).withColumn(
    'BOROUGH', 
    upper(col('BOROUGH'))).withColumn(
    'ON STREET NAME', upper(col('ON STREET NAME'))).withColumn(
    'VEHICLE TYPE CODE 1', upper(col('VEHICLE TYPE CODE 1')))

# Create functions for time of day and season
def time_of_day(hour, minute):
    return when((hour == 0) & (minute >= 0) | ((hour >= 1) & (hour <= 6)), 'SMALL HOURS') \
        .when((hour > 6) & (hour <= 12), 'MORNING') \
        .when((hour > 12) & (hour <= 18), 'AFTERNOON') \
        .otherwise('EVENING')

def time_of_year(month):
    return when((month >= 3) & (month <= 5), 'SPRING') \
        .when((month >= 6) & (month <= 8), 'SUMMER') \
        .when((month >= 9) & (month <= 11), 'AUTUMN') \
        .otherwise('WINTER')

#Apply functions to create new columns

df = df.withColumn('TIME OF DAY', 
              time_of_day(hour(df['CRASH TIME']),minute(df['CRASH TIME']))
             ).withColumn('SEASON', time_of_year(month(df['CRASH DATE'])))


# # Organize columns

df = df.select('DATE', 'CRASH DATE', 'CRASH TIME', 'TIME OF DAY', 'SEASON',
               'BOROUGH', 'ZIP CODE', 'LATITUDE', 'LONGITUDE', 'ON STREET NAME', 
               'NUMBER OF PERSONS INJURED', 'NUMBER OF PERSONS KILLED',
               'CONTRIBUTING FACTOR VEHICLE 1', 'VEHICLE TYPE CODE 1')

print('Count of rows: {0}'.format(df.count()), '\n')

df.show(5)
df.printSchema()

+-------------------+----------+----------+-----------+------+---------+--------+----------+-----------+--------------------+-------------------------+------------------------+-----------------------------+--------------------+
|               DATE|CRASH DATE|CRASH TIME|TIME OF DAY|SEASON|  BOROUGH|ZIP CODE|  LATITUDE|  LONGITUDE|      ON STREET NAME|NUMBER OF PERSONS INJURED|NUMBER OF PERSONS KILLED|CONTRIBUTING FACTOR VEHICLE 1| VEHICLE TYPE CODE 1|
+-------------------+----------+----------+-----------+------+---------+--------+----------+-----------+--------------------+-------------------------+------------------------+-----------------------------+--------------------+
|2012-12-01 00:00:00|2012-12-01|      0:00|SMALL HOURS|WINTER|MANHATTAN|   10013|40.7233076|  -74.00298|BROOME STREET    ...|                        0|                       0|         AGGRESSIVE DRIVIN...|               OTHER|
|2012-12-01 00:05:00|2012-12-01|      0:05|SMALL HOURS|WINTER|MANHATTAN|   10036|40.7572

### DataFrame reduction by Time of Day

In [6]:
df_time_of_day = df

In [7]:
# Group the data by CRASH DATE and TIME OF DAY
grouped_time_of_day = df_time_of_day.groupBy("CRASH DATE", "TIME OF DAY")

# Aggregate the grouped data
counts_time_of_day = grouped_time_of_day.agg(
    count("CRASH TIME").alias("NUMBER OF CRASHES"),
    sum("NUMBER OF PERSONS INJURED").alias("PERSONS INJURED"),
    sum("NUMBER OF PERSONS KILLED").alias("PERSONS KILLED"),
    mode("LATITUDE").alias("LATITUDE"),
    mode("LONGITUDE").alias("LONGITUDE"),
    mode("BOROUGH").alias("BOROUGH"),
    mode("ON STREET NAME").alias("ON STREET NAME"),
    mode("CONTRIBUTING FACTOR VEHICLE 1").alias("CONTRIBUTING FACTOR VEHICLE 1"),
    mode("VEHICLE TYPE CODE 1").alias("VEHICLE TYPE CODE 1"),
    mode("SEASON").alias("SEASON")
)

counts_time_of_day = counts_time_of_day.withColumnRenamed("CRASH DATE", "DATE") \
                   .withColumnRenamed("BOROUGH", "PEAK BOROUGH") \
                   .withColumnRenamed("ON STREET NAME", "PEAK STREET") \
                   .withColumnRenamed("CONTRIBUTING FACTOR VEHICLE 1", "MAIN FACTOR") \
                   .withColumnRenamed("VEHICLE TYPE CODE 1", "MAIN VEHICLE TYPE")

# Convert TIME OF DAY column to categorical and sort accordingly
time_of_day_org = ['SMALL HOURS', 'MORNING', 'AFTERNOON', 'EVENING']
counts_time_of_day = counts_time_of_day.withColumn("TIME OF DAY", F.when(col("TIME OF DAY").isNull(), "UNKNOWN").otherwise(col("TIME OF DAY")))
counts_time_of_day = counts_time_of_day.withColumn("TIME OF DAY", F.when(col("TIME OF DAY").isin(time_of_day_org), col("TIME OF DAY")).otherwise("OTHER"))
counts_time_of_day = counts_time_of_day.withColumn("TIME OF DAY", counts_time_of_day["TIME OF DAY"].cast("string"))

# Sort the DataFrame by DATE and TIME OF DAY
counts_time_of_day = counts_time_of_day.orderBy("DATE", F.when(col("TIME OF DAY") == "SMALL HOURS", 1)
                                                               .when(col("TIME OF DAY") == "MORNING", 2)
                                                               .when(col("TIME OF DAY") == "AFTERNOON", 3)
                                                               .when(col("TIME OF DAY") == "EVENING", 4)
                                                               .otherwise(5))


# Organize columns

counts_time_of_day.select('DATE', 'TIME OF DAY', 'SEASON', 'NUMBER OF CRASHES',
                          'PEAK STREET', 'PEAK BOROUGH', 
                          'LATITUDE', 'LONGITUDE','PERSONS INJURED', 
                          'PERSONS KILLED','MAIN FACTOR', 
                          'MAIN VEHICLE TYPE')



# Rename df

df_time_of_day = counts_time_of_day

df_time_of_day.show(5)
#print('Count of rows: {0}'.format(df_time_of_day.count()), '\n')


+----------+-----------+-----------------+---------------+--------------+----------+-----------+------------+--------------------+-----------+-----------------+------+
|      DATE|TIME OF DAY|NUMBER OF CRASHES|PERSONS INJURED|PERSONS KILLED|  LATITUDE|  LONGITUDE|PEAK BOROUGH|         PEAK STREET|MAIN FACTOR|MAIN VEHICLE TYPE|SEASON|
+----------+-----------+-----------------+---------------+--------------+----------+-----------+------------+--------------------+-----------+-----------------+------+
|2012-12-01|SMALL HOURS|               74|             22|             0|40.7569869|-73.7375192|   MANHATTAN|NORTHERN BOULEVAR...|UNSPECIFIED|PASSENGER VEHICLE|WINTER|
|2012-12-01|    MORNING|              100|             20|             0|40.8261757|-73.9508609|    BROOKLYN|FLATBUSH AVENUE  ...|UNSPECIFIED|PASSENGER VEHICLE|WINTER|
|2012-12-01|  AFTERNOON|              139|             32|             0|40.6673324|-73.8631562|    BROOKLYN|FOREST AVENUE    ...|UNSPECIFIED|PASSENGER VEHICLE|

### DataFrame reduction by Day

In [8]:
df_day = df_time_of_day

In [10]:
# Group the data by DATE

grouped_day = df_day.groupBy("DATE")

# Aggregate the grouped data
counts_day = grouped_day.agg(
    sum("NUMBER OF CRASHES").alias("NUMBER OF CRASHES"),
    mode("SEASON").alias("SEASON"),
    sum("PERSONS INJURED").alias("PERSONS INJURED"),
    sum("PERSONS KILLED").alias("PERSONS KILLED"),
    mode("LATITUDE").alias("LATITUDE"),
    mode("LONGITUDE").alias("LONGITUDE"),
    mode("PEAK BOROUGH").alias("PEAK BOROUGH"),
    mode("PEAK STREET").alias("PEAK STREET"),
    mode("MAIN FACTOR").alias("MAIN FACTOR"),
    mode("MAIN VEHICLE TYPE").alias("MAIN VEHICLE TYPE")
)

# Rename df

df_day = counts_day

print('Count of rows: {0}'.format(df_day.count()), '\n')

df_day.show(5)
df.printSchema()

Count of rows: 4126 



+----------+-----------------+------+---------------+--------------+----------+-----------+------------+--------------------+-----------+-----------------+
|      DATE|NUMBER OF CRASHES|SEASON|PERSONS INJURED|PERSONS KILLED|  LATITUDE|  LONGITUDE|PEAK BOROUGH|         PEAK STREET|MAIN FACTOR|MAIN VEHICLE TYPE|
+----------+-----------------+------+---------------+--------------+----------+-----------+------------+--------------------+-----------+-----------------+
|2012-12-01|              383|WINTER|            100|             0|40.7569869|-73.7375192|   MANHATTAN|EAST HOUSTON STRE...|UNSPECIFIED|PASSENGER VEHICLE|
|2012-12-02|              352|WINTER|             92|             0| 40.759626|-73.9954735|   MANHATTAN|WEST 57 STREET   ...|UNSPECIFIED|PASSENGER VEHICLE|
|2012-12-03|              471|WINTER|            111|             0|40.8416419|-73.8982589|    BROOKLYN|LINDEN BOULEVARD ...|UNSPECIFIED|PASSENGER VEHICLE|
|2012-12-04|              464|WINTER|            135|           

## DataFrame reduction by Year

In [11]:
df_year = df_day

In [12]:
# Group the data by DATE

grouped_year = df_year.withColumn("DATE", year("DATE")).groupBy("DATE")
# df.withColumn('TIME OF DAY', 
#               time_of_day(hour(df['CRASH TIME']),

# Aggregate the grouped data
counts_year = grouped_year.agg(
    sum("NUMBER OF CRASHES").alias("NUMBER OF CRASHES"),
    mode("SEASON").alias("SEASON"),
    sum("PERSONS INJURED").alias("PERSONS INJURED"),
    sum("PERSONS KILLED").alias("PERSONS KILLED"),
    mode("LATITUDE").alias("LATITUDE"),
    mode("LONGITUDE").alias("LONGITUDE"),
    mode("PEAK BOROUGH").alias("PEAK BOROUGH"),
    mode("PEAK STREET").alias("PEAK STREET"),
    mode("MAIN FACTOR").alias("MAIN FACTOR"),
    mode("MAIN VEHICLE TYPE").alias("MAIN VEHICLE TYPE")
)

# Sort by DATE

counts_year.orderBy('DATE')

# Rename df

df_year = counts_year

print('Count of rows: {0}'.format(df_year.count()), '\n')

df_year.show()
df_year.printSchema()

Count of rows: 13 



+----+-----------------+------+---------------+--------------+----------+-----------+------------+--------------------+-----------+-----------------+
|DATE|NUMBER OF CRASHES|SEASON|PERSONS INJURED|PERSONS KILLED|  LATITUDE|  LONGITUDE|PEAK BOROUGH|         PEAK STREET|MAIN FACTOR|MAIN VEHICLE TYPE|
+----+-----------------+------+---------------+--------------+----------+-----------+------------+--------------------+-----------+-----------------+
|2018|            92574|SPRING|          27746|           112|   40.7171|  -73.75764|      QUEENS|LINDEN BOULEVARD ...|UNSPECIFIED|            SEDAN|
|2015|           159799|SPRING|          37327|           177|40.7184302|-73.9928041|    BROOKLYN|LINDEN BOULEVARD ...|UNSPECIFIED|PASSENGER VEHICLE|
|2023|            38260|SPRING|          24962|           136|  40.65945| -74.000534|      QUEENS| WOODHAVEN BOULEVARD|UNSPECIFIED|            SEDAN|
|2022|            39439|SPRING|          22717|           128|         0|          0|      QUEENS| W

## HADDOP directory writing csv files 
* Can be created (written) only once, after that over function must be used

In [19]:
# df_time_of_day.write.option("header", True).csv('/user1/df_time_of_day')
# df_day.write.option("header", True).csv('/user1/df_day')
# df_year.write.options(header='True', delimiter=',').csv('/user1/df_year')